## Introduction

### Problem Statement
Patients often struggle to find reliable information about medicines for their symptoms, leading to confusing and inappropriate self-medication. Traditional search methods on the internet may return unreliable results, while medical databases are often too technical for general users.

This project presents a simple but advanced AI-powered medicine recommendation system that helps users find appropriate medications based on their symptoms or health conditions. Built using natural language processing and retrieval-augmented generation (RAG) technologies, the system provides accurate and comprehensive medicine information based on the provided dataset with information of 11,000 medications.

## Installing libraries in colab environment

In [ ]:
%%capture --no-stderr
%pip install numpy==1.24.3 transformers==4.37.2 --no-cache-dir

In [ ]:
%%capture --no-stderr
%pip install langchain-community langchain-openai langchain-chroma langchain-huggingface crewai sentence-transformers gradio xmltodict git+https://github.com/PrithivirajDamodaran/Gramformer.git

* These libraries are temporary installed in colab environment and needs to be installed every time you run this notebook.

* To correctly run this notebook, run two cells above and click restart and run all.

In [ ]:
# Connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import necessary libraries
import os
import getpass
import json
import torch
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_chroma.vectorstores import Chroma
from langchain.memory import ConversationBufferMemory
import crewai
from crewai.tools import tool, BaseTool
from crewai import Agent, Task, Crew, Process, LLM
from pydantic import BaseModel, Field
from typing import Type
from gramformer import Gramformer
import inspect
from langchain_community.retrievers import PubMedRetriever
import gradio as gr

In [ ]:
# Set base path in google drive
base_path = '/content/drive/MyDrive/Project/Project/DEMO_APP_Medicine_info/notebooks'
os.chdir(base_path)

In [ ]:
# Setting environment variables for APIs and other configurations
os.environ["LANGSMITH_TRACING"] = "true"
os.environ['USER_AGENT'] = 'Mozilla/5.0'
os.environ['OPENAI_API_KEY'] = getpass.getpass()
os.environ['LANGSMITH_API_KEY'] = getpass.getpass()
os.environ['HF_TOKEN'] = getpass.getpass()
os.environ["NCBI_API_KEY"] = getpass.getpass()

··········
··········
··········
··········


* API keys successfully retrieved.

In [ ]:
# Use embedding model saved in google drive
embedding = HuggingFaceEmbeddings(
    model_name = '/content/drive/MyDrive/Project/Project/DEMO_APP_Medicine_info/embedding_model',
    encode_kwargs = {'batch_size': 64,
                     'normalize_embeddings': True},
)

In [ ]:
# Reads the JSON file and stores its contents
file_path = '/content/drive/MyDrive/Project/Project/DEMO_APP_Medicine_info/data/medi_list.json'
with open(file_path, 'r') as f:
  data = json.load(f)

In [ ]:
# Extract each item from the data to create a document object and add it to a list.
documents = []
for entry in data:
    documents.append(Document(
        page_content = entry["Uses"].strip(),
        metadata = {
            "Medicine Name": entry["Medicine Name"],
            "Composition": entry["Composition"],
            "Side Effects": entry["Side_effects"],
            "Image URL": entry["Image URL"],
            "Manufacturer": entry["Manufacturer"],
            "Excellent Review %": entry["Excellent Review %"],
            "Average Review %": entry["Average Review %"],
            "Poor Review %": entry["Poor Review %"]
        }
    ))

In [ ]:
# Check the document
print(documents[0].page_content)
print(documents[0].metadata)

Cancer of colon and rectum Non-small cell lung cancer Kidney cancer Brain tumor Ovarian cancer Cervical cancer
{'Medicine Name': 'Avastin 400mg Injection', 'Composition': 'Bevacizumab (400mg)', 'Side Effects': 'Rectal bleeding Taste change Headache Nosebleeds Back pain Dry skin High blood pressure Protein in urine Inflammation of the nose', 'Image URL': 'https://onemg.gumlet.io/l_watermark_346,w_480,h_480/a_ignore,w_480,h_480,c_fit,q_auto,f_auto/f5a26c491e4d48199ab116a69a969be3.jpg', 'Manufacturer': 'Roche Products India Pvt Ltd', 'Excellent Review %': 22, 'Average Review %': 56, 'Poor Review %': 22}


In [ ]:
# Set up a vector database with documents and embedding model
vector_db = Chroma.from_documents(documents = documents, embedding = embedding)

In [ ]:
# Check the vector database is working properly
vector = embedding.embed_query('fever')
vector_db.similarity_search_by_vector(vector)

[Document(id='32725c97-289b-42d0-ba6a-e53faa355a29', metadata={'Poor Review %': 31, 'Average Review %': 29, 'Composition': 'Paracetamol (300mg) + Paracetamol (700mg)', 'Manufacturer': 'Torrent Pharmaceuticals Ltd', 'Image URL': 'https://onemg.gumlet.io/l_watermark_346,w_480,h_480/a_ignore,w_480,h_480,c_fit,q_auto,f_auto/cropped/hkwnne0rswjlobsul6pj.jpg', 'Side Effects': 'No common side effects seen', 'Excellent Review %': 40, 'Medicine Name': 'XTPara Proglet Tablet SR'}, page_content='Pain relief Fever'),
 Document(id='a837cf75-a4d3-4756-bfab-a8cc6e9ac976', metadata={'Excellent Review %': 0, 'Poor Review %': 100, 'Composition': 'Paracetamol (300mg) + Paracetamol (700mg)', 'Average Review %': 0, 'Side Effects': 'No common side effects seen', 'Manufacturer': 'Unichem Laboratories Ltd', 'Medicine Name': 'Xtpara Tablet SR', 'Image URL': 'https://onemg.gumlet.io/l_watermark_346,w_480,h_480/a_ignore,w_480,h_480,c_fit,q_auto,f_auto/cropped/eeqkaal8jfnecy1k5bem.jpg'}, page_content='Pain relief

* The vector database successfully retrieved information on the pain killers related to fever.

In [ ]:
# Convert the vector database into retriever for semantic search
retriever = vector_db.as_retriever()

In [ ]:
# Large language model for AI task
llm = LLM(model = 'gpt-4o', temperature = 0.0)

In [ ]:
# Replace occurrences of 'use_auth_token = ' with 'token = '
gf_path = inspect.getfile(Gramformer)

with open(gf_path, "r") as f:
    lines = f.readlines()

with open(gf_path, "w") as f:
    for line in lines:
        if "use_auth_token=" in line:
            # Replace use_auth_token with token
            line = line.replace("use_auth_token=", "token=")
        f.write(line)

In [ ]:
# Initialize Gramformer with model 1 (grammar correction)
gf = Gramformer(models=1)

# Define input schema for the grammar correction tool
class CorrectionInput(BaseModel):
  # Required field for user input text to be corrected
  user_input: str = Field(..., description = 'User input (text) that may need grammar or spelling corrections')

# Define the grammar correction tool using BaseTool
class GFCorrectionTool(BaseTool):
  # Name of the tool
  name: str = 'Spell & Grammar Corrector'

  # Description of what the tool does
  description: str = """
                     Check whether the user input needs to be corrected with spelling and grammar.
                     Otherwise, keep it unchanged.
                     """
  # Specify the expected input schema
  args_schema: Type[BaseModel] = CorrectionInput

  # Define the main logic of the tool
  def _run(self, user_input: str) -> str:
    try:
      # Get corrected version of the input text (only 1 best candidate)
      corrections = list(gf.correct(user_input, max_candidates=1))

      # If correction differs from input, return it; otherwise, return original input
      if corrections and corrections[0] != user_input:
        return corrections[0]
      else:
        return user_input

    # Handle exceptions with error message
    except Exception as e:
      return e

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:902: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

[Gramformer] Grammar error correct/highlight model loaded..


In [ ]:
# Initialize fallback retriever for PubMed
pubmed_retriever = PubMedRetriever()

# Define input schema for the medicine retriever tool
class Retriever(BaseModel):
  # Required field: user symptom or keyword to search
  query: str = Field(..., description='Symptom or keyword to search for appropriate medicine.')

# Define the medicine retriever tool using BaseTool
class MedicineRetrieverTool(BaseTool):
  # Name of the tool
  name: str = 'Medicine Retriever'

  # Description of what this tool does
  description: str = 'Finds medicines based on symptoms using the vector store. Falls back to PubMed if needed.'

  # Expected input schema
  args_schema: Type[BaseModel] = Retriever

  # Core logic of the tool
  def _run(self, query: str) -> str:
    try:
      # Try retrieving results from the main retriever
      results = retriever.invoke(query)

      # If results are found, format and return the top 3
      if results:
        formatted = []
        for doc in results[:3]:
          meta = doc.metadata
          formatted.append(
              f"Use: {doc.page_content}\n"
              f"Medicine Name: {meta.get('Medicine Name')}\n"
              f"Manufacturer: {meta.get('Manufacturer')}\n"
              f"Composition: {meta.get('Composition')}\n"
              f"Side Effects: {meta.get('Side Effects')}\n"
              f"Reviews: {meta.get('Excellent Review %')}% Excellent / {meta.get('Poor Review %')}% Poor\n"
              f"Image URL: {meta.get('Image URL')}"
          )
        return '\n\n'.join(formatted)

      else:
        # If no internal results, fallback to PubMed
        pubmed_results = pubmed_retriever.invoke(query)

        # If PubMed also fails, return no results message
        if not pubmed_results:
          return "No medicine found in our internal database or PubMed."

        # Format and return top 3 PubMed results
        formatted_pubmed = []
        for doc in pubmed_results[:3]:
          formatted_pubmed.append(
              f"PubMed Result:\n{doc.page_content}..."
          )

        return "No internal match found. Here are some PubMed results:\n\n" + '\n\n'.join(formatted_pubmed)

    # Handle unexpected errors during retrieval
    except Exception as e:
      return f"Retrieval failed: {str(e)}"

In [ ]:
# Create the memory object
memory = ConversationBufferMemory(return_messages = False)
history = memory.load_memory_variables({})["history"]

# Define input schema for answer generation
class Generator(BaseModel):
  # Required field: structured context containing medicine information
  context: str = Field(..., description='Information on the Medicine')

# Define the tool that generates human-readable answers from structured info
class AnswerGeneratorTool(BaseTool):
  # Name of the tool
  name: str = "Answer Generator"

  # Description of the tool’s purpose
  description: str = "Generates a natural-language answer based on raw medicine information."

  # Expected input schema
  args_schema: Type[BaseModel] = Generator

  # Main function that generates the output
  def _run(self, context: str) -> str:
    try:
        # Create a prompt instructing the language model to summarize the information
        prompt = (
            "You are an expert in writing a summarized paragraph using the retrieved information. "
            "Based on the following structured information, generate a brief paragraph about the medicine"
            "that explains the medicine's name, use, manufacturer, composition, side effects, and reviews. "
            "Do NOT include the Image URL to your answer."
            "Write clearly and be concise.\n\n"
            "If the retrieved medicine is more than 1, write them in seperate paragraphs with clear labels."
            "Use markdown-style formatting with bold labels and line breaks for better readability.\n\n"
            f"Chat History:\n{history}\n\n"
            f"New Context:\n{context}\n\n"
            )

        # Generate a response using the language model
        response = llm.call(prompt)
        return response

    # Handle any errors in generation process
    except Exception as e:
        return f"Answer generation failed: {str(e)}"

<ipython-input-20-3722907723>:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages = False)


In [ ]:
# Define input schema for hallucination checking
class HallucinationInput(BaseModel):
  # Required: original context used for generation
  context: str = Field(..., description='Structured source information')
  # Required: LLM-generated answer to be checked
  generated_answer: str = Field(..., description='LLM-generated answer based on the context')

# Tool that detects and corrects hallucinated content in an LLM-generated answer
class HallucinationCheckerTool(BaseTool):
  # Name of the tool
  name: str = "Hallucination Checker"

  # Tool description
  description: str = "Checks the generated answer against the context. If hallucination is found, regenerate the answer."

  # Define expected input schema
  args_schema: Type[BaseModel] = HallucinationInput

  # Core logic for hallucination detection and correction
  def _run(self, context: str, generated_answer: str) -> str:
    try:
      # Prompt to verify if the generated answer contains unsupported claims
      check_prompt = (
        "You are an expert in comparing the context and generated answer. "
        "Your task is to determine whether the following generated answer "
        "contains any hallucinated or unsupported information when compared to the base context.\n\n"
        "Context (source info):\n"
        f"{context}\n\n"
        "Generated Answer:\n"
        f"{generated_answer}\n\n"

        "Respond with a short assessment. If all claims are supported, say 'No hallucination detected.' "
        "Otherwise, say 'Hallucination found.' Explain which parts are inaccurate or unsupported."
      )

      # Ask the language model to check for hallucinations
      result = llm.call(check_prompt)

      # If no hallucination is found, return original answer
      if "No hallucination detected" in result:
        return generated_answer
      else:
        # If hallucination is found, regenerate the answer strictly based on context
        fix_prompt = (
          "You are an expert in writing grounded and factual medical summaries. "
          "Based strictly on the following context, generate a revised answer that avoids any hallucinations. "
          "Be clear, concise, and accurate. Do not include any information not present in the context.\n\n"
          f"{context}"
        )
        revised_answer = llm.call(fix_prompt)
        return revised_answer

    # Catch and report any runtime errors
    except Exception as e:
      return f"Hallucination check failed: {str(e)}"

In [ ]:
# Agent for correcting grammar and spelling issues
gf_agent = Agent(
    role = "Spelling & Grammar Corrector",  # Describes the agent's job
    goal = "Fix all spelling and grammar mistakes in the user's input.",  # Defines what the agent should achieve
    tools = [GFCorrectionTool()],  # Uses the GFCorrectionTool defined earlier
    verbose = True,  # Enables detailed logging for debugging
    backstory = "An expert in interpreting user input and fixing natural language errors to ensure clear communication."
)

# Agent for retrieving medicine info based on user symptoms
retriever_agent = Agent(
    role = "Medicine Finder",
    goal = "Retrieve medicines relevant to the user's symptoms or concerns.",  # Main objective
    backstory = (
        "You are a helpful assistant trained to search a curated medical vector database. "
        "You return clean, detailed information about matching medicines based on the user's query."
    ),
    tools = [MedicineRetrieverTool()],  # Uses the medicine retriever tool
    verbose = True
)

# Agent for generating natural language summaries from structured medicine data
answer_agent = Agent(
    role = "Medical Answer Generator",
    goal = "Generate clear, concise summaries of medicine information using structured data.",
    tools = [AnswerGeneratorTool()],  # Uses the answer generation tool
    verbose = True,
    backstory = (
        "You are a medical writer who specializes in converting raw data into easy-to-understand summaries. "
        "You only use the provided structured context to create short, informative paragraphs."
    )
)

# Agent for verifying factual accuracy and correcting hallucinated output
hallucination_checker_agent = Agent(
    role = "Hallucination Detector",
    goal = "Detect any hallucinated or unsupported claims in AI-generated summaries.",  # Main goal
    tools = [HallucinationCheckerTool()],  # Uses the hallucination checker tool
    verbose = True,
    backstory = (
        "You are a strict factual verifier tasked with ensuring all content is backed by the provided data. "
        "If any unsupported information is found, you require a revision grounded strictly in the original source."
    )
)

In [ ]:
def run_medicine_assistant_manual(user_input: str) -> str:

    # Grammar and spelling correction
    gf_tool = gf_agent.tools[0]
    corrected = gf_tool._run(user_input)

    # Retrieve relevant medicine info based on corrected input
    med_tool = retriever_agent.tools[0]
    meds = med_tool._run(corrected)

    # Generate a natural-language summary of the retrieved medicine info
    summary = answer_agent.tools[0]._run(meds)

    # Check the summary for hallucinations and fix if necessary
    final = hallucination_checker_agent.tools[0]._run(meds, summary)

    # Update memory
    memory.save_context({"input": user_input}, {"output": final})

    return f"Suggestions for {user_input.capitalize()}:\n\n{final}"

In [ ]:
demo = gr.Interface(
    fn = run_medicine_assistant_manual,
    inputs = gr.Textbox(label = "Symptom or Disease",
                        placeholder = "Enter symptom or disease (e.g., fever, nausea, diarrhea)"),
    outputs = gr.Textbox(label = "Medicine Info", lines = 10),
    title = "💊 Medicine Information Assistant",
    description = "Type a symptom or disease name to receive recommended medicine information based on your input.",
    flagging_mode = "never"
)

In [ ]:
demo.launch(share = True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9a0179ab45e80110c1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
